In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy import stats
import joblib

Cargar datos

In [2]:
# Cargar el DataFrame vuelos
df_vuelos = pd.read_csv('Filghts TEC_Valid.csv')

# Eliminar los duplicados por 'Flight_ID' y quedarte con el último registro
df_vuelos = df_vuelos.drop_duplicates(subset='Flight_ID', keep='last')

Preparar df

In [3]:
# Filtrar datos eliminando registros con valores nulos en las columnas especificadas
df_vuelos = df_vuelos.dropna(subset=['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type'])

# Convertir la columna 'STD' a formato datetime
df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
df_vuelos['STA'] = pd.to_datetime(df_vuelos['STA'])
df_vuelos['Departure_WeekDay'] = df_vuelos['STD'].dt.day_name()

# Extraer la hora y el mes de la columna 'STD'
df_vuelos['Departure_Hour'] = df_vuelos['STD'].dt.hour
df_vuelos['Arrival_Hour'] = df_vuelos['STA'].dt.hour
df_vuelos['Departure_Month'] = df_vuelos['STD'].dt.month
# Calcular la duración del vuelo restando STA de STD
df_vuelos['Flight_Time'] = (pd.to_datetime(df_vuelos['STA']) - pd.to_datetime(df_vuelos['STD'])).dt.total_seconds() / 60

# Convertir el tiempo de vuelo a entero
df_vuelos['Flight_Time'] = df_vuelos['Flight_Time'].astype(int)
# Convertir la hora de salida y el mes a entero
df_vuelos['Departure_Hour'] = df_vuelos['Departure_Hour'].astype(int)
df_vuelos['Departure_Month'] = df_vuelos['Departure_Month'].astype(int)

# Crear año para luego filtrar
df_vuelos['Departure_Year'] = df_vuelos['STD'].dt.year

# Eliminar columnas 'STD', 'STA' y 'Bookings'
df_vuelos.drop(['STD', 'STA', 'Aeronave', 'Bookings', 'Destination_Type', 'Origin_Type'], axis=1, inplace=True)

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Codificar las columnas categóricas
categorical_columns = ['DepartureStation', 'ArrivalStation', 'Departure_WeekDay']
for column in categorical_columns:
    df_vuelos[column] = label_encoder.fit_transform(df_vuelos[column])
    
# Head del DataFrame
df_vuelos.head()

C:\Users\RHJ\AppData\Local\Temp\ipykernel_28976\1047186869.py:5: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_vuelos['STD'] = pd.to_datetime(df_vuelos['STD'])
C:\Users\RHJ\AppData\Local\Temp\ipykernel_28976\1047186869.py:6: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_vuelos['STA'] = pd.to_datetime(df_vuelos['STA'])


,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Flight_Time,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Departure_Year
0,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023
1,efd86c996035dacdca7a0ccb2560dda1,34,17,186,197.0,240,1,0,4,7,2023
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,18,14,220,NaN,45,0,17,17,2,2024
3,dd0fad3248951d2f71d63e6279aeaa4b,18,14,220,200.0,40,1,15,15,6,2023
5,3b5df8805161ea827d2f2e4298c38e06,6,13,240,183.0,55,4,17,18,9,2023


Predecir registros nulos

In [5]:
# Filtrar los registros con valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos_prediccion = df_vuelos[df_vuelos['Passengers'].isnull()]
# Filtrar los registros sin valores nulos en las columnas 'Passengers' y 'Bookings'
df_vuelos = df_vuelos.dropna(subset=['Passengers'])

print("Cantidad de vuelos predectivos: ", len(df_vuelos_prediccion))
print("Cantidad de vuelos de entrenamiento: ", len(df_vuelos))

# Eliminar registros con valores nulos en la columna 'Passengers' del DataFrame original
df_vuelos_prediccion = df_vuelos_prediccion.drop(columns=['Passengers'])

# Flight ID's vuelos predictivos
flight_ids = df_vuelos_prediccion['Flight_ID']
flight_ids['Departure_Year'] = df_vuelos_prediccion['Departure_Year'] 

# Eliminar columna 'Flight_ID'
df_vuelos_prediccion.drop(columns=['Flight_ID'], inplace=True)
df_vuelos_prediccion.drop(columns=['Departure_Year'], inplace=True)

# Cargar el modelo desde un archivo
modelo = joblib.load('modelo_pasajeros.pkl')

# Predecir la cantidad de pasajeros
predicciones = modelo.predict(df_vuelos_prediccion)

# Asignar las predicciones al DataFrame de vuelos predictivos
df_vuelos_prediccion['Passengers'] = predicciones

# Asignar los Flight ID's a las predicciones
df_vuelos_prediccion['Flight_ID'] = flight_ids
df_vuelos_prediccion['Departure_Year'] = flight_ids['Departure_Year']

# Unir los DataFrames
df_vuelos = pd.concat([df_vuelos, df_vuelos_prediccion])

# Remover dataframes temporales
del df_vuelos_prediccion

# Cantidad de vuelos en el DataFrame
print("Cantidad de vuelos en el DataFrame: ", len(df_vuelos))
# head del DataFrame
df_vuelos.head()


Cantidad de vuelos predectivos:  0
Cantidad de vuelos de entrenamiento:  119151


C:\Users\RHJ\AppData\Local\Temp\ipykernel_28976\2994148727.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_ids['Departure_Year'] = df_vuelos_prediccion['Departure_Year']
c:\Users\RHJ\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ValueError: node array from the pickle has an incompatible dtype:
- expected: [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]
- got     : {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}

Df ventas filtrado

In [ ]:
# Cargar el DataFrame especificando los tipos de datos
df_ventas = pd.read_csv('Sales TEC_Valid.csv', dtype={'Quantity': int})

# Lista de cadenas para filtrar
categorias_mantener = ['Botanas', 'Galletas', 'Bebidas Calientes', 'Refrescos', 'Sopas', 'Perecederos']

# Filtrar el DataFrame y eliminar la columna 'TotalSales'
df_ventas = df_ventas[df_ventas['ProductType'].isin(categorias_mantener)].drop(columns=['TotalSales'])

# Eliminar la columna 'ProductType'
df_ventas.drop(columns=['ProductType'], inplace=True)

# Verificar y eliminar duplicados en el DataFrame original
df_ventas = df_ventas.drop_duplicates(subset=['Flight_ID', 'ProductName'])

# Obtener todos los nombres de productos únicos
productos_unicos = df_ventas['ProductName'].unique()

# Crear DataFrame con todas las combinaciones posibles de Flight_ID y ProductName
flight_ids = df_ventas['Flight_ID'].unique()
combinaciones = []
for flight_id in flight_ids:
    for producto in productos_unicos:
        combinaciones.append({'Flight_ID': flight_id, 'ProductName': producto})
df_combinaciones = pd.DataFrame(combinaciones)

# Fusionar con el DataFrame original para obtener las combinaciones faltantes
df_faltantes = pd.merge(df_combinaciones, df_ventas, on=['Flight_ID', 'ProductName'], how='left')

# Filtrar solo las combinaciones faltantes
df_faltantes = df_faltantes[df_faltantes['Quantity'].isnull()]
df_faltantes['Quantity'] = 0

# Concatenar las combinaciones faltantes al DataFrame principal
df_ventas = pd.concat([df_ventas, df_faltantes], ignore_index=True)

# Verificar y eliminar duplicados en el DataFrame combinado
df_ventas = df_ventas.drop_duplicates(subset=['Flight_ID', 'ProductName'])

# Ordenar el DataFrame por Flight_ID y ProductName
df_ventas = df_ventas.sort_values(by=['Flight_ID', 'ProductName']).reset_index(drop=True)

# Resetear los índices del DataFrame
df_ventas.reset_index(drop=True, inplace=True)

# Cantidad de registros en el DataFrame
print("Cantidad de registros en el DataFrame: ", len(df_ventas))

Cantidad de registros en el DataFrame:  6045572


Generar df de vuelos completos con productos

In [ ]:
# Obtener todos los productos únicos de df_ventas
productos_unicos = df_ventas['ProductName'].unique()

# Crear un DataFrame con todos los vuelos
df_vuelos_completo = pd.DataFrame({'Flight_ID': df_vuelos['Flight_ID'].unique()})

# Crear un DataFrame con todos los productos únicos
df_productos = pd.DataFrame({'ProductName': productos_unicos})

# Hacer una combinación de cada Flight_ID con cada producto único
df_vuelos_completo = pd.merge(df_vuelos_completo, df_productos, how='cross')

# Agregar la columna Quantity con valor 0
df_vuelos_completo['Quantity'] = 0

# Resetear los índices del DataFrame
df_vuelos_completo.reset_index(drop=True, inplace=True)

# Cantidad de registros en el DataFrame
print("Cantidad de registros en el DataFrame: ", len(df_vuelos_completo))

# Mostrar las primeras filas del DataFrame resultante
df_vuelos_completo.head(20)

Cantidad de registros en el DataFrame:  9326690


,Flight_ID,ProductName,Quantity
0,ab954014077430bd842cfa305a55c0f8,Arandano,0
1,ab954014077430bd842cfa305a55c0f8,Arandano Mango Mix,0
2,ab954014077430bd842cfa305a55c0f8,Arcoiris,0
3,ab954014077430bd842cfa305a55c0f8,Cafe 19 Cafe Clasico,0
4,ab954014077430bd842cfa305a55c0f8,Cafe 19 Capuchino,0
5,ab954014077430bd842cfa305a55c0f8,Cafe 19 Chiapas,0
6,ab954014077430bd842cfa305a55c0f8,Cafe Costa,0
7,ab954014077430bd842cfa305a55c0f8,Cafe De Olla,0
8,ab954014077430bd842cfa305a55c0f8,Carne Seca Habanero,0
9,ab954014077430bd842cfa305a55c0f8,Carne Seca Original,0


Completar todas las ventas

In [ ]:
# Concatenar df_ventas y df_vuelos_completo
df_ventas = pd.concat([df_ventas, df_vuelos_completo], ignore_index=True)

# Cantidad de registros en el DataFrame
print("Cantidad de registros en el DataFrame: ", len(df_ventas))

# Mostrar el resultado
df_ventas.head(100)

Cantidad de registros en el DataFrame:  15372262


,Flight_ID,ProductName,Quantity
0,00004a718edba9d9ef878d08f02ae057,Arandano,0
1,00004a718edba9d9ef878d08f02ae057,Arandano Mango Mix,0
2,00004a718edba9d9ef878d08f02ae057,Arcoiris,0
3,00004a718edba9d9ef878d08f02ae057,Cafe 19 Cafe Clasico,0
4,00004a718edba9d9ef878d08f02ae057,Cafe 19 Capuchino,0
...,...,...,...
95,0000cd79c0c3a9c309df6064dcacaeea,Nissin Res,0
96,0000cd79c0c3a9c309df6064dcacaeea,Nueces De Arbol Mix,0
97,0000cd79c0c3a9c309df6064dcacaeea,Nutty Berry Mix,0
98,0000cd79c0c3a9c309df6064dcacaeea,Panini Clasico,0


Inner join de toda la información

In [ ]:
# Realizar el inner join entre los DataFrames
df_vuelos_ventas = pd.merge(df_vuelos, df_ventas, on='Flight_ID', how='inner')

# Cantidad de registros en el DataFrame resultante
print("Cantidad de registros en el DataFrame: ", len(df_vuelos_ventas))

# Mostrar los primeros 100 registros del DataFrame resultante
df_vuelos_ventas.head(100)


Cantidad de registros en el DataFrame:  15109406


,Flight_ID,DepartureStation,ArrivalStation,Capacity,Passengers,Flight_Time,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,Departure_Year,ProductName,Quantity
0,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Arandano,0
1,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Arandano Mango Mix,0
2,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Arcoiris,0
3,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Cafe 19 Cafe Clasico,0
4,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Cafe 19 Capuchino,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Nissin Res,0
96,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Nueces De Arbol Mix,0
97,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Nutty Berry Mix,0
98,ab954014077430bd842cfa305a55c0f8,15,21,240,229.0,165,4,11,14,10,2023,Panini Clasico,0


In [ ]:
'''
Hay ventas que no tienen registros de vuelo, por eso hay perdida de registros.

# Cargar el DataFrame vuelos
df_vuelos = pd.read_csv('Filghts TEC_Valid.csv')
# Cargar el DataFrame especificando los tipos de datos
df_ventas = pd.read_csv('Sales TEC_Valid.csv', dtype={'Quantity': int})

# Eliminar los duplicados por 'Flight_ID' y quedarte con el último registro
df_vuelos = df_vuelos.drop_duplicates(subset='Flight_ID', keep='last')

# Encuentra los 'Flight_ID' en df_ventas que no están en df_vuelos
valores_no_en_vuelos = df_ventas[~df_ventas['Flight_ID'].isin(df_vuelos['Flight_ID'])]

# Cuenta el número de valores únicos de 'Flight_ID' que no están en df_vuelos
num_valores_no_en_vuelos = valores_no_en_vuelos['Flight_ID'].nunique()
print("Número de valores de 'Flight_ID' en df_ventas que no están en df_vuelos:", num_valores_no_en_vuelos)

# Número de valores de 'Flight_ID' en df_ventas que no están en df_vuelos: 4378
'''

'\nHay ventas que no tienen registros de vuelo, por eso hay perdida de registros.\n\n# Cargar el DataFrame vuelos\ndf_vuelos = pd.read_csv(\'Filghts TEC_Valid.csv\')\n# Cargar el DataFrame especificando los tipos de datos\ndf_ventas = pd.read_csv(\'Sales TEC_Valid.csv\', dtype={\'Quantity\': int})\n\n# Eliminar los duplicados por \'Flight_ID\' y quedarte con el último registro\ndf_vuelos = df_vuelos.drop_duplicates(subset=\'Flight_ID\', keep=\'last\')\n\n# Encuentra los \'Flight_ID\' en df_ventas que no están en df_vuelos\nvalores_no_en_vuelos = df_ventas[~df_ventas[\'Flight_ID\'].isin(df_vuelos[\'Flight_ID\'])]\n\n# Cuenta el número de valores únicos de \'Flight_ID\' que no están en df_vuelos\nnum_valores_no_en_vuelos = valores_no_en_vuelos[\'Flight_ID\'].nunique()\nprint("Número de valores de \'Flight_ID\' en df_ventas que no están en df_vuelos:", num_valores_no_en_vuelos)\n\n# Número de valores de \'Flight_ID\' en df_ventas que no están en df_vuelos: 4378\n'

Encodear df

In [ ]:
df_vuelos_ventas['ProductName'] = label_encoder.fit_transform(df_vuelos_ventas['ProductName'])
df_vuelos_ventas.head()
df_vuelos_ventas.drop(columns=['Flight_ID'], inplace=True)

Separamos lo real de lo creado

In [ ]:
# Separar cantidad de productos vendidos iguales a 0
df_vuelos_ventas_prediccion = df_vuelos_ventas[(df_vuelos_ventas['Departure_Year'] == 2024) | (df_vuelos_ventas['Departure_Year'] == 2025)]
df_vuelos_ventas = df_vuelos_ventas[(df_vuelos_ventas['Departure_Year'] != 2024) & (df_vuelos_ventas['Departure_Year'] != 2025)]  
df_vuelos_ventas_prediccion.drop(columns=['Departure_Year'], inplace=True)  
df_vuelos_ventas.drop(columns=['Departure_Year'], inplace=True) 

# Cantidad de registros en el DataFrame de ventas predicción
print("Cantidad de registros en el DataFrame de ventas predicción: ", len(df_vuelos_ventas_prediccion))
# Cantidad de registros en el DataFrame de ventas
print("Cantidad de registros en el DataFrame de ventas: ", len(df_vuelos_ventas))
df_vuelos_ventas.head(30)

Cantidad de registros en el DataFrame de ventas predicción:  2416106
Cantidad de registros en el DataFrame de ventas:  12693300


,DepartureStation,ArrivalStation,Capacity,Passengers,Flight_Time,Departure_WeekDay,Departure_Hour,Arrival_Hour,Departure_Month,ProductName,Quantity
0,15,21,240,229.0,165,4,11,14,10,0,0
1,15,21,240,229.0,165,4,11,14,10,1,0
2,15,21,240,229.0,165,4,11,14,10,2,0
3,15,21,240,229.0,165,4,11,14,10,3,0
4,15,21,240,229.0,165,4,11,14,10,4,1
5,15,21,240,229.0,165,4,11,14,10,5,0
6,15,21,240,229.0,165,4,11,14,10,6,0
7,15,21,240,229.0,165,4,11,14,10,7,0
8,15,21,240,229.0,165,4,11,14,10,8,0
9,15,21,240,229.0,165,4,11,14,10,9,1


Creación de modelo

In [ ]:
# Dividir los datos en características (X) y la variable objetivo (y)
X = df_vuelos_ventas.drop(columns=['Quantity'])
y = df_vuelos_ventas['Quantity']  # Ahora y es un DataFrame con dos columnas

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=577)

# Definir el rango de hiperparámetros para la búsqueda de cuadrícula
param_grid = {
    'n_estimators': [50, 100], 
    'learning_rate': [0.05, 0.1], 
    'max_depth': [3, 4], 
    'min_samples_split': [2],
    'min_samples_leaf': [1], 
} 

# Inicializar el modelo de Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor(random_state=577)

# Inicializar GridSearchCV con el modelo y los hiperparámetros
grid_search = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid, cv=2, scoring='neg_mean_absolute_percentage_error', n_jobs=-1, verbose=3, error_score='raise')

# Entrenar GridSearchCV en los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)


Fitting 2 folds for each of 8 candidates, totalling 16 fits


Resultados de predicción

In [ ]:
# Obtener el mejor modelo encontrado
best_model = grid_search.best_estimator_

y_true = np.array(y_test)
y_pred = best_model.predict(X_test)
df_predicciones = pd.DataFrame({'Real': y_true, 'Predicción': y_pred}).round(0)
df_predicciones.head(20)

# Calcular la diferencia porcentual de pasajeros entre las predicciones y los valores reales
df_predicciones['Diferencia porcentual de ventas'] = abs((df_predicciones['Real'] - df_predicciones['Predicción']) / df_predicciones['Real'])*100

# Mostrar las primeras filas del DataFrame de predicciones
df_predicciones.describe()

Observar características valiosas

In [ ]:
# Obtener la importancia de las características
feature_importance = best_model.feature_importances_

# Crear un DataFrame con la importancia de las características
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importance})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Mostrar el ranking de importancia de características
print(feature_importance_df)

Guardar modelo

In [ ]:
# Guardar el modelo en un archivo
joblib.dump(best_model, 'modelo_ventas.pkl')